In [0]:
# Databricks Notebook source
from pyspark.sql.functions import *
from pyspark.sql.window import Window

# Configure Path
SILVER_PATH = "s3a://data-files-rjx/silver/fire_calls"
GOLD_PATH = "s3a://data-files-rjx/gold/fire_metrics"

def create_gold_layer():
    try:
        # 1. Read Silver layer data
        silver_df = spark.read.format("delta").load(SILVER_PATH)
        
        # 2. Core Index calculation
        metrics_df = (silver_df
            .groupBy(
            year("Call Date").alias("call_year"),
            month("Call Date").alias("call_month"),
            "Call Type",
            "Station Area"
        )
            .agg(
                count("*").alias("total_incidents"),
        )
            .withColumn("report_date", current_date())
        )
        
        # 3. Time table extraction
        date_dim_df = (silver_df
            .select(
                year("Call Date").alias("year"),
                month("Call Date").alias("month"),
                dayofmonth("Call Date").alias("day"),
                date_format("Call Date", "EEEE").alias("weekday")
            )
            .distinct()
        )
        
        # 4. Write into Gold layer
        (metrics_df.write
            .mode("overwrite")
            .format("delta")
            .option("delta.columnMapping.mode", "name")
            .partitionBy("call_year")  # By year
            .save(f"{GOLD_PATH}/fact_incidents"))
        
        (date_dim_df.write
            .mode("overwrite")
            .format("delta")
            .save(f"{GOLD_PATH}/dim_date"))
        
        # 5. Registry metadata
        spark.sql(f"""
            CREATE TABLE IF NOT EXISTS gold_fact_incidents
            USING DELTA
            LOCATION '{GOLD_PATH}/fact_incidents'
        """)
        
        print("✅ Gold层聚合完成！")
        return True
        
    except Exception as e:
        print(f"❌ 聚合失败: {str(e)}")
        raise


create_gold_layer()

In [0]:
%sql
select * from gold_fact_incidents
limit 50